In [1]:
import matplotlib
from matplotlib import pyplot as plt
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# import matplotlib_inline
# %config InlineBackend.figure_format='retina'


# load data

In [159]:
t2m_era5 = xr.open_mfdataset('/data0/samuelb/como2024/ERA5/T2m/balkans/ERA5_????_T2m_*.nc').var167
t2m_mpi = xr.open_mfdataset('/data0/samuelb/como2024/MPI-ESM1-2-HR/tas/balkans/*_historical_*.nc').tas
sm_land_era5 = xr.open_mfdataset('/data0/samuelb/como2024/ERA5/swvl1_land/balkans/ERAland_????_swvl1_*.nc').var39
# sm_era5 = xr.open_mfdataset('/data0/samuelb/como2024/ERA5/swvl1/balkans/ERA5_????_swvl1_*.nc').var39
# sm_mpi = xr.open_mfdataset('/data0/samuelb/como2024/MPI-ESM1-2-HR/mrsos/*_balkans_*_historical_*.nc').mrsos


In [163]:
landmask = sm_land_era5.isel(time=0)!=9.96921e+36
t2m_era5 = t2m_era5.where(landmask)
# sm_era5 = sm_era5.where(landmask)
sm_land_era5 = sm_land_era5.where(landmask)
# sm_mpi = sm_mpi.where(sm_mpi!=9.96921e+36)/100


In [198]:
t2m_era5_summer = t2m_era5.where((t2m_era5.time.dt.month==6) | (t2m_era5.time.dt.month==7) | (t2m_era5.time.dt.month==8))
sm_land_era5_summer = sm_land_era5.where((sm_era5.time.dt.month==6) | (sm_land_era5.time.dt.month==7) | (sm_land_era5.time.dt.month==8))


# calculate hot / dry days

In [205]:
# calculate heat threshold
t2m_era5_summer_95p = t2m_era5_summer.chunk({'time':-1}).quantile(.95,'time')

# calculate dry threshold
sm_land_era5_summer_05p = sm_land_era5_summer.chunk({'time':-1}).quantile(.05,'time')


In [206]:
# select heat excededance days
t2m_era5_summer_exceed = t2m_era5_summer > t2m_era5_summer_95p

# select dry exceedance days
sm_land_era5_summer_exceed = sm_land_era5_summer < sm_land_era5_summer_05p


In [216]:
# sum of exceedance days per week
hot_days = t2m_era5_summer_exceed.resample(time='1W').sum()
dry_days = sm_land_era5_summer_exceed.resample(time='1W').sum()
hot_dry_days = (t2m_era5_summer_exceed & sm_land_era5_summer_exceed).resample(time='1W').sum()


/home/samuelb/.conda/envs/como2024/lib/python3.9/site-packages/xarray/groupers.py:326: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
/home/samuelb/.conda/envs/como2024/lib/python3.9/site-packages/xarray/groupers.py:326: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
